# Terrain-following coordinates

Many problems in the earth sciences are thin-film flows, i.e. the characteristic length-scale $L$ of the domain is much larger than the characteristic thickness $H$.
We can then describe the geometry in terms of a *footprint* or *base* domain $\Omega$ which lives in 1D or 2D.
We then imagine that we know
1. some *bed elevation* field $b$, which could be, say, the topography of the land surface or the bathymetry of the ocean floor, and
2. a *thickness* field $h$ describing the vertical extent of the medium we care about.

For example, ocean flow, glacier flow, lava flow, and rainfall runoff over a landscape are all thin-film flows.
(I'm excluding atmospheric circulation because how you define a thickness field isn't quite as obvious.)
We could discretize the geometry, toss it into a mesh generator, and then solve our problem using the finite element method.
But what if the thickness field is changing in time, so that we have an evolving free surface?
We would not only have to keep track of the evolution of, say, the velocity field inside the triangulation, but the triangulation itself would be changing in time in a non-trivial way.

Terrain-following coordinates instead let us do all of our computing on the Cartesian product $\Omega \times [0, H]$ of the footprint domain and an interval.
We then warp or deform this computational domain onto the physical domain.
The basal topography and variable thickness of the true physical domain are "encoded" into the coordinate transformation, which works its way into the variational form.
While the physical geometry might change in time, the computational geometry does not, and so we have a much simpler way of working with free surface flows.

#### The transformation

I'm going to derive all of this for 2 + 1D problems and you can do the appropriate simplifications for 1 + 1D ones.
We'll write $x_1, x_2, x_3$ for the coordinates of the physical domain, where $x_1$, $x_2$ are the horizontal coordinates and $x_3$ the vertical coordinate.
We start knowing the bed elevation field $b(x_1, x_2)$ and the thickness field $h(x_1, x_2)$.
The *terrain-following coordinates* $\xi_1$, $\xi_2$, $\xi_3$ are defined as
$$\begin{align}
\xi_1 & = x_1 \\
\xi_2 & = x_2 \\
\xi_3 & = H\frac{x_3 - b(x_1, x_2)}{h(x_1, x_2)}
\end{align}$$
where $H$ is a vertical scale that we can pick.
In the physical domain, $x_3$ could take values between $b(x_1, x_2)$ and $b(x_1, x_2) + h(x_1, x_2)$, and this range changes with position in space.
In terrain-following coordinates, $\xi_3$ can only take values between 0 and $H$.

#### Derivatives

Now suppose that the problem we wanted to solve was posed as variational form on the physical domain:
$$\int_\Omega\int_b^{b + h}\left\{\partial_tG(q)\cdot\phi - F(q, \nabla_xq, \ldots)\cdot\nabla_x\phi - S(q, \ldots)\cdot\phi\right\}dx_3\,dx_2\,dx_1 = 0$$
for all test functions $\phi$.
How can we transform this into a variational form defined on the computational domain?
We'll first have to transform derivatives defined with respect to $x$ into derivatives with respect to $\xi$.

One key observation.
When we take a partial derivative of some field with respect to $x_1$, we do so with with the vertical coordinate $x_3$ held fixed.
But when we take a partial derivative of a field with respect to $\xi_1$, we do so with the *relative depth* $\xi_3$ held fixed, and those two are not the same thing because of variations in the surface and bed topography.

If we're talking about derivatives of the bed elevation and thickness, however, those fields only depend on the horizontal coordinates, which are unchanged.
Consequently we can unambigously write $\nabla b$ or $\nabla h$ without having to specify which coordinate system we're working with.

In order to transform derivatives of fields, we need to first calculate the derivative of the transformation itself, also known as the Jacobian:
$$\frac{\partial\xi}{\partial x} \equiv \left[\begin{matrix}\frac{\partial\xi_1}{\partial x_1} & \frac{\partial\xi_1}{\partial x_2} & \frac{\partial\xi_1}{\partial x_3} \\ \frac{\partial\xi_2}{\partial x_1} & \frac{\partial\xi_2}{\partial x_2} & \frac{\partial\xi_2}{\partial x_3} \\ \frac{\partial\xi_3}{\partial x_1} & \frac{\partial\xi_3}{\partial x_2} & \frac{\partial\xi_3}{\partial x_3} \end{matrix}\right].$$
I'm going to introduce a new quantity here that's going to save a lot of math writing.
The *geometric distortion factor* $\gamma$ is the following vector:
$$\gamma = -\frac{H}{h}\left(\nabla b + \xi_3\nabla h\right)$$
We can then calculate explicitly that
$$\frac{\partial\xi}{\partial x} = \left[\begin{matrix}I & 0 \\ \gamma^\top & H/h\end{matrix}\right]$$
where $I$ is the 2x2 identity matrix.
It's worth working through this yourself.

#### Volume distortion

First, we can work out how volumes get distorted and how this changes the integrals we need to compute.
The change of variables formula states that
$$dx_3\,dx_2\,dx_1 = \left|\det\frac{d\xi}{dx}\right|^{-1}d\xi_3\,d\xi_2,d\xi_1\ldots$$
and from the expression above we can calculate right away that this is:
$$\ldots = \frac{h}{H}d\xi_3\,d\xi_2\,d\xi_1$$
You can remember this by thinking about the case where we take $H = 1$ to be a dimensionless constant.
The differential volume element should have units of volume.

#### Gradient distortion

Now we can start asking ourselves how derivatives change.
This is just the chain rule:
$$\frac{\partial\phi}{\partial x} = \frac{\partial\phi}{\partial\xi}\frac{\partial\xi}{\partial x}$$
**There is a lurking subtlety here.**
Derivatives start life as *row* vectors and not as column vectors.
It's only when we think of a derivative as a row vector that the chain rule applies as straightforwardly for multidimensional problems as it does in single-variable calculus.
The gradient of a scalar field, on the other hand, is a column vector which we obtain by taking the transpose of this row vector.
Silly right?
But when there are serious coordinate transformations, it means that **we need to take the transpose of the Jacobian**:
$$\nabla_x\phi = \frac{\partial\xi}{\partial x}^\top\nabla_\xi\phi.$$
This is profoundly strange, unless of course you've taken general relativity in which case you've already been broken.

### The domain

First, we'll create our computational domain -- a regular mesh of the unit square with quadrilateral elements.

In [ ]:
import firedrake
from firedrake import Constant, inner, grad, dot, dx, ds
import numpy as np
import matplotlib.pyplot as plt

nx, nz = 32, 32
computational_domain = firedrake.UnitSquareMesh(nx, nz, quadrilateral=True)

Our physical domain will be obtained by warping the coordinates, like we saw in the thermochronometry notebook.
Here I've wrapped up some of the functions to create the bed topography because we'll have to do so more than once.

In [ ]:
δb = Constant(1 / 4)
α = Constant(1 / 8)
ξ_0 = Constant(1 / 2)

def sech(z):
    return 2 / (firedrake.exp(z) + firedrake.exp(-z))

def bed_topography(ξ):
    return δb * sech((ξ - ξ_0) / α)

In [ ]:
ξ = firedrake.SpatialCoordinate(computational_domain)
bed_expr = bed_topography(ξ[0])
surf_expr = Constant(1.0)
expr = firedrake.as_vector((ξ[0], (1 - ξ[1]) * bed_expr + ξ[1] * surf_expr))

Vc = computational_domain.coordinates.function_space()
X = firedrake.Function(Vc)
X.interpolate(expr)
physical_domain = firedrake.Mesh(X, reorder=False)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(computational_domain, axes=ax)
ax.legend();

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(physical_domain, axes=ax)
ax.legend();

### Steady diffusion

We can work out some of the hard parts by focusing first on a steady-state diffusion problem.
If we compute a temperature field on the physical domain and on the computational domain, we should get the same answer.
In order to illustrate to you the importance of getting the metric terms correct, I'm going to do things the wrong way.

I'll use $q$ for the field we're solving for (temperature, hydraulic head, etc).
The variational form that I'll use for the code below is
$$\int_\Omega \left\{k\nabla q\cdot\nabla\phi - fq\right\}g\,dx = 0$$
for all test functions $\phi$ that are zero on the boundary, together with a fixed value boundary condition $q|_{\partial\Omega} = q_{\text{ext}}$.
Here $k$ is the conductivity and $g$ is a scalar field that can weight the integral differently in space.
I haven't used this before but it'll represent the volume distortion derived above.
We could have factored it into the source terms and conductivity, but it's arguably preferable not to do that.
I'll also note here that, in previous problems, I've assumed that the conductivity $k$ is a scalar.
We could also take $k$ to be a matrix here, in which case the variational form, written out in index notation, would be
$$\int_\Omega\left\{k_{ij}\partial_jq\cdot\partial_i\phi - fq\right\}g\,dx = 0.$$
As I'll show below, **the diffusion equation in terrain-following coordinates looks like the diffusion equation in Cartesian coordinates, but with a tensorial conductivity.**

First, I'm going to write a generic function that will solve the diffusion equation.
The arguments are what function space we're solving in, the conductivity tensor $k$, and a volume distortion factor $g$.

In [ ]:
bottom_ids = (3,)
top_ids = (4,)

def solve_diffusion(fn_space, k, g):
    q = firedrake.Function(fn_space)
    ϕ = firedrake.TestFunction(fn_space)
    F = inner(dot(k, grad(q)), grad(ϕ)) * g * dx
    bottom_bc = firedrake.DirichletBC(fn_space, 0, bottom_ids)
    top_bc = firedrake.DirichletBC(fn_space, 1, top_ids)
    firedrake.solve(F == 0, q, [bottom_bc, top_bc])
    return q

Now we'll create the ground truth solution defined on the physical domain.
Here the conductivity tensor is a constant multiple of the identity matrix and the volume distortion factor is a constant.

In [ ]:
cg1 = firedrake.FiniteElement("CG", "quadrilateral", 1)
Q_physical = firedrake.FunctionSpace(physical_domain, cg1)

k = Constant(1.0)
g = Constant(1.0)
I = firedrake.Identity(2)

q_physical = solve_diffusion(Q_physical, k * I, g)

Now let's make a solution on the computational domain which gets the right volume distortion factor, but which doesn't account for any distortion of the gradients -- we're still using a multiple of the identity matrix as our conductivity tensor.

In [ ]:
Q_computational = firedrake.FunctionSpace(computational_domain, cg1)

k = Constant(1.0)
ξ = firedrake.SpatialCoordinate(computational_domain)
b = bed_topography(ξ[0])
h = surf_expr - b
H = Constant(1.0)

q_bad = solve_diffusion(Q_computational, k * I, h / H)

I can compare the two solutions by manually transferring the raw data that lives inside `q_bad` into a Function defined on the physical domain.

In [ ]:
q_bad_transfer = firedrake.Function(Q_physical)
q_bad_transfer.dat.data[:] = q_bad.dat.data_ro[:]

We can see by looking at the contour lines of the ground truth solution and the conjured up bad solution that the two are not equal.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
levels = np.linspace(0, 1.0, 11)
firedrake.tricontour(q_physical, levels=levels, colors="tab:blue", axes=ax)
firedrake.tricontour(q_bad_transfer, levels=levels, colors="tab:orange", linestyles="dotted", axes=ax);

Here I'm using a function to create the Jacobian $\partial\xi/\partial x$ as a tensor explicitly using the formula we wrote down above.
Now we can compute that the inside of the variational form is:
$$\begin{align}
k\nabla_xq\cdot\nabla_x\phi & = k\left(\frac{\partial\xi}{\partial_x}^\top\nabla_\xi q\right)\cdot\frac{\partial\xi}{\partial_x}^\top\nabla_\xi\phi \\
& = k\nabla_\xi q\cdot\frac{\partial\xi}{\partial x}\frac{\partial\xi}{\partial x}^\top\nabla_\xi\phi.
\end{align}$$
Now we can hoist out the *metric tensor*
$$m_{ij} = \frac{\partial\xi_i}{\partial x_k}\frac{\partial\xi_j}{\partial x_k}$$
which is realized below using the function `firedrake.dot`.
The product of the scalar conductivity and the metric tensor gives the tensor conductivity that we use in terrain-following coordinates.

In [ ]:
γ = -H * (b.dx(0) + ξ[1] * h.dx(0)) / h
dξ_dx = firedrake.as_tensor(
    [
        [1.0, 0.0],
        [γ, H / h],
    ]
)

m = dot(dξ_dx, dξ_dx.T)

When we solve the problem again but with the correct factors of both the metric tensor and the volume distortion factor, everything lines up perfectly.

In [ ]:
q_tfc = solve_diffusion(Q_computational, k * m, h / H)

In [ ]:
q_tfc_transfer = firedrake.Function(Q_physical)
q_tfc_transfer.dat.data[:] = q_tfc.dat.data_ro[:]

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
levels = np.linspace(0, 1.0, 11)
firedrake.tricontour(q_physical, levels=levels, colors="tab:blue", axes=ax)
firedrake.tricontour(q_tfc_transfer, levels=levels, colors="tab:orange", linestyles="dotted", axes=ax);

Moral of the story: solving the diffusion equation in terrain-following coordinates looks just like doing it in Cartesian coordinates, but there's an anisotropic conductivity and its preferred directions are determined by the warping of space.

### Advection + diffusion

With the pure diffusion problem, we had to confront (1) the volumetric distortion term and (2) the gradient distortion terms.
When we include advection, we also have to think about how vector fields transform.
Vector fields don't transform the same as gradients!
To see why this is so, let's write $u_x$ and $u_\xi$ for the expressions of the same vector field in Cartesian and terrain-following coordinates.
A vector field can be thought of as the trajectory of a point $x$, so
$$u_x = \frac{dx}{dt}$$
and likewise $u_\xi = d\xi/dt$.
But we can again apply the chain rule to express $u_x$ in terms of $u_\xi$:
$$\begin{align}
u_x & = \frac{dx}{dt} \\
 & = \frac{dx}{d\xi}\frac{d\xi}{dt} \\
 & = \left(\frac{d\xi}{dx}\right)^{-1}\frac{d\xi}{dt} \\
 & = \left(\frac{d\xi}{dx}\right)^{-1}u_\xi
\end{align}$$
This is different from how we transformed gradients!
The rules:
* To transform a **gradient**, multiply on the left by $d\xi/dx^\top$
* To transform a **vector field**, multiply on the left by $(d\xi/dx)^{-1}$.

Now let's suppose we were solving an advection-diffusion equation instead:
$$\int_\Omega\left(\partial_tq\cdot\phi - qu_x\cdot\nabla_x\phi + k\nabla_xq\cdot\nabla_x\phi\right)dx = 0$$
for all test functions $\phi$.
We already know how to rewrite the diffusive part in $\xi$-coordinates.
I'm going to write $J$ for the Jacobian $d\xi/dx$.
For the advective part, we can compute
$$\begin{align}
u_x\cdot\nabla_x\phi & = \left(J^{-1}u_\xi\right)\cdot J^\top\nabla_\xi\phi \\
& = u_\xi\cdot J^{-\top}J^\top\nabla_\xi\phi \\
& = u_\xi\cdot\nabla_\xi\phi.
\end{align}$$
In other words, the factors of the Jacobian all cancel out!

We will need to be able to compute the inverse of the Jacobian if we're to map the velocity field in Cartesian coordinates to the velocity in terrain-following coordinates.
Luckily this is pretty easy to do by calculating the inverse of the transformation itself:
$$\begin{align}
x_1 & = \xi_1 \\
x_2 & = \xi_2 \\
x_3 & = b(\xi_1, \xi_2) + \frac{h(\xi_1, \xi_2)}{H}\xi_3
\end{align}$$
so we can compute directly that
$$\begin{align}
\frac{dx}{d\xi} = \left[\begin{matrix}I & 0 \\ \nabla b^\top + \xi_3\nabla h^\top/H & h/H\end{matrix}\right].
\end{align}$$
It's worth thinking about the velocity field at a point on the bottom boundary assuming that a wind field tangential to the boundary.
In Cartesian coordinates, the wind field has a non-zero vertical component, but in terrain-following coordinates it doesn't.

#### Making a velocity field

Once again, I'll make a divergence-free velocity field by taking a velocity potential $\Psi$ and setting
$$u_x = \nabla_x\times\Psi = \left(\begin{matrix}-\partial_{x_2}\Psi \\ +\partial_{x_1}\Psi\end{matrix}\right).$$
If $\Psi$ is zero on the boundary, then the gradient of $\Psi$ points away from the boundary and thus its curl is tangential to the boundary.
We can further guarantee that $u$ is equal to zero on the boundary by taking $\Psi = \Upsilon^2$ for some function $\Upsilon$ that is zero on the boundary.
The code below creates the velocity potential and the velocity and plots them both.

In [ ]:
x = firedrake.SpatialCoordinate(physical_domain)

expr = (x[0] * (1 - x[0]) * (1 - x[1]) * (x[1] - bed_topography(x[0]))) ** 2
cg = firedrake.FiniteElement("CG", "quadrilateral", 1)
Ψ = firedrake.Function(Q_physical).interpolate(expr)

In [ ]:
fig, ax = plt.subplots()
colors = firedrake.tripcolor(Ψ, axes=ax)
fig.colorbar(colors);

In [ ]:
grad_Ψ = firedrake.grad(expr)
V_physical = firedrake.VectorFunctionSpace(physical_domain, cg)
u_expr = firedrake.as_vector((-grad_Ψ[1], grad_Ψ[0]))
u_x = firedrake.Function(V_physical).interpolate(u_expr)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.quiver(u_x, axes=ax)
fig.colorbar(colors);

#### Transforming the velocity

Now if we want to solve the advection-diffusion equation in terrain-following coordinates, we'll need to transform the velocity field.
The formula I wrote above expressed the Cartesian velocity field $u_x$ as $J^{-1}u_\xi$, but instead I started with the Cartesian velocity field and now I want the terrain-following one.
This is just
$$u_\xi = Ju_x.$$
We can implement this by transferring the raw data for the velocity field in Cartesian coordinates over the computational domain and then applying the inverse of the Jacobian pointwise.
We could have also done this with the velocity potential but I won't.

You might actually have to do this if you wanted to run a simulating in terrain-following coordinates but you got some velocity data in Cartesian coordinates from, say, a reanalysis product.

In [ ]:
V_computational = firedrake.VectorFunctionSpace(computational_domain, cg)
u_transfer = firedrake.Function(V_computational)
u_transfer.dat.data[:] = u_x.dat.data_ro[:]
u_ξ = firedrake.Function(V_computational).interpolate(dot(dξ_dx, u_transfer))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.quiver(u_ξ, axes=ax)
fig.colorbar(colors);

### Variational form and solution

Now let's repeat the same exercise as above.
Putting everything together, the variational form that we want to solve is
$$\int_\Omega\left\{\partial_tq\cdot\phi - qu_\xi\cdot\nabla_\xi\phi + km\nabla_\xi q\cdot\nabla_\xi\phi\right\}g\,dx = 0$$

Here I'm going to set a different diffusion coefficient than the value I used before.
I want to make the Péclet number large enough that the problem is advection-dominated.
(Ask me what happens when there's no diffusion at all...)

In [ ]:
max_speed = np.abs(u_x.dat.data_ro).flatten().max()
domain_size = 1.0
peclet_number = 500.0

diffusion_coefficient = max_speed * domain_size / peclet_number
print(f"Max speed:             {max_speed:.5e}")
print(f"Diffusion coefficient: {diffusion_coefficient:.5e}")
k = Constant(diffusion_coefficient)

In [ ]:
import irksome
from irksome import Dt
import tqdm

In [ ]:
x_0 = firedrake.Constant((0.25, 0.55))
r = firedrake.Constant(0.25)
q_expr = firedrake.exp(-inner(x - x_0, x - x_0) / r**2)

q_physical = firedrake.Function(Q_physical)
q_physical.interpolate(q_expr)

q_computational = firedrake.Function(Q_computational)
q_computational.dat.data[:] = q_physical.dat.data_ro[:]

In [ ]:
final_time = 2 * domain_size / max_speed
min_cell_size = physical_domain.cell_sizes.dat.data_ro.min()
timestep = min_cell_size / max_speed / 8
num_steps = int(final_time / timestep)

print(f"Final time: {final_time:.2f}")
print(f"Timestep:   {timestep:.2f}")

Once again, I'm wrapping up the important stuff in a function that I can reuse.
The `degree=4` bit at the end of the form is to use a lower-order quadrature rule; without this, Firedrake will try to estimate the quadrature degree and will pick something ridiculous.

In [ ]:
def solve_advection_diffusion(q_0, u, k, g, timestep, num_steps):
    t = Constant(0.0)
    dt = Constant(timestep)

    Q = q_0.function_space()
    q = q_0.copy(deepcopy=True)
    ϕ = firedrake.TestFunction(Q)
    F = (Dt(q) * ϕ - q * inner(u, grad(ϕ)) + inner(dot(k, grad(q)), grad(ϕ))) * g * dx(degree=4)
    method = irksome.BackwardEuler()
    solver = irksome.TimeStepper(F, method, t, dt, q)
    
    qs = [q.copy(deepcopy=True)]
    for step in tqdm.trange(num_steps):
        solver.advance()
        qs.append(q.copy(deepcopy=True))

    return qs

In [ ]:
qs_physical = solve_advection_diffusion(q_physical, u_x, k * I, Constant(1.0), timestep, num_steps)
qs_computational = solve_advection_diffusion(q_computational, u_ξ, k * m, h / H, timestep, num_steps)

Now we can transfer all the results using terrain-following coordinates over onto the physical domain.

In [ ]:
q_transfer = firedrake.Function(Q_physical)
qs_transferred = []
for q in qs_computational:
    q_transfer.dat.data[:] = q.dat.data_ro[:]
    qs_transferred.append(q_transfer.copy(deepcopy=True))

And finally make a movie of them both.

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
for ax in axes:
    ax.set_aspect("equal")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

axes[0].set_title("Physical")
axes[1].set_title("Terrain-following")

kw = {"vmin": 0.0, "vmax": 1.0, "num_sample_points": 4}
colors_physical = firedrake.tripcolor(qs_physical[0], axes=axes[0], **kw)
colors_computational = firedrake.tripcolor(qs_transferred[0], axes=axes[1], **kw)

fn_plotter = firedrake.FunctionPlotter(physical_domain, num_sample_points=4)
def animate(pair):
    q_phys, q_comp = pair
    colors_physical.set_array(fn_plotter(q_phys))
    colors_computational.set_array(fn_plotter(q_comp))

animation = FuncAnimation(fig, animate, zip(qs_physical, qs_transferred), interval=1e3/20)

In the eyeball norm they look about the same.

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

In [ ]:
differences = np.array(
    [
        firedrake.assemble(abs(q_1 - q_2) * dx)
        for q_1, q_2 in zip(qs_physical, qs_transferred)
    ]
)

If we plot the relative errors, they remain below 0.2% throughout.

In [ ]:
times = np.linspace(0, final_time, num_steps + 1)
fig, ax = plt.subplots()
ax.set_xlabel("time")
ax.set_ylabel("relative difference")
ax.plot(times, differences / firedrake.assemble(q_expr * dx));